In [2]:
!pip install kagglehub

   ---------------------------------------- 0.0/51.9 kB ? eta -:--:--
   ----------------------- ---------------- 30.7/51.9 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 51.9/51.9 kB 661.7 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import kagglehub

data_dir = kagglehub.dataset_download("andrewmvd/dog-cat-detection")
print("Path to dataset file: ", data_dir)